In [1]:
! pip install pandas numpy matplotlib

In [3]:
import glob
import pandas as pd

In [8]:
file_paths = glob.glob('dataset_020/color_dataset*.txt')
dfs = []

In [9]:
for file_path in file_paths:
    df = pd.read_csv(file_path, delimiter=',')
    dfs.append(df)

In [11]:
final_df = pd.concat(dfs, ignore_index=True)

In [293]:
color_df = pd.read_csv('dataset_020/color_dataset01.txt', sep=',', header=None, usecols=[1,2,3,4,5], names=["control_value", "R", "G", "B", "W"] )

In [294]:
color_df.iloc[990:]

,control_value,R,G,B,W
990,0.993,1009,1329,1496,3852
991,0.994,1008,1330,1497,3853
992,0.995,1008,1330,1497,3852
993,0.996,1007,1330,1499,3854
994,0.997,1006,1330,1499,3854
995,0.998,1006,1330,1499,3854
996,0.999,1006,1331,1500,3855
997,0.000,1121,1171,1199,3454
998,0.001,1322,882,660,2728
999,0.002,1322,883,661,2731


In [295]:
def correct_dataset(df):
    """
    Correct the dataset based on provided rules.
    
    df: The dataframe
    
    Returns the corrected dataframe.
    """
    
    # For the start of the dataset
    first_faulty_index = df['W'].diff().abs().idxmax()
    for idx in range(first_faulty_index):
        next_index = first_faulty_index + 1
        if next_index in df.index:  # Check if index exists in DataFrame
            for col in ['R', 'G', 'B', 'W']:
                df.at[idx, col] = int(df.loc[next_index, col])
            df.at[idx, 'control_value'] = (idx) * 0.001 + 0.001
        if (df.control_value != 0.000).any():
            zero_row = df.iloc[0].to_frame().T
            zero_row['control_value'] = 0.000
            df = pd.concat([zero_row, df]).reset_index(drop=True)
            
    
    # For the end of the dataset
    last_faulty_index = df['W'].iloc[::-1].diff().abs().idxmax()
    for idx in range(last_faulty_index, len(df)):
        lower_control_value = df.loc[idx-1, 'control_value']
        if df.at[idx, 'control_value'] != lower_control_value - 0.001:
            df = df.loc[:idx-1]
            break
        
        prev_index = last_faulty_index - 1
        if prev_index in df.index:  # Check if index exists in DataFrame
            for col in ['R', 'G', 'B', 'W']:
                df.at[idx, col] = int(df.loc[prev_index, col])
            df.at[idx, 'control_value'] = (idx) * 0.001 + 0.001
    
    last_row = df.iloc[-1].to_frame().T
    last_row['control_value'] = 1.000
    df = pd.concat([df, last_row]).reset_index(drop=True)
    return df

In [296]:
corrected_df = correct_dataset(color_df)

In [298]:
corrected_df.iloc[:10]

,control_value,R,G,B,W
0,0.000,1330.0,894.0,670.0,2759.0
1,0.001,1330.0,894.0,670.0,2759.0
2,0.002,1332.0,896.0,672.0,2765.0
3,0.003,1330.0,894.0,670.0,2759.0
4,0.004,1329.0,893.0,670.0,2756.0
5,0.005,1329.0,892.0,669.0,2755.0
6,0.006,1329.0,892.0,669.0,2755.0
7,0.007,1328.0,892.0,670.0,2755.0
8,0.008,1328.0,893.0,671.0,2756.0
9,0.009,1328.0,893.0,671.0,2756.0
